In [2]:
%load_ext autoreload
%autoreload 2

In [273]:
import argiope, compmod2
import pandas as pd
import numpy as np
from string import Template

In [274]:
class CuboidSample(argiope.models.Part):
    """
    A cuboid test sample.
    """
    def __init__(self, shape = (1, 1, 1), dim = (1., 1., 1.)):
        self.dim = dim
        self.shape = shape

    def make_mesh(self):
        """
        Makes the mesh
        """    
        mesh = argiope.mesh.structured_mesh(dim = self.dim, shape = self.shape)
        self.mesh = mesh

In [275]:
'''
def get_node_pairs(pair, normal = "x"):
    """
    Returns pairs of nodes that are in the same position on two faces modulo
    a translation along a normal direction.
    """
    plane = ["x", "y", "z"]
    plane = [d for d in plane if d != normal]
    pair = [left, right]
    pair = [d.coords[plane].sort_values(plane) for d in pair]
    return np.array([d.index for d in pair]).T        

get_node_pairs(pair = [front, back] , normal = "z").shape'''
0

0

In [276]:
main_template = """
********************************************************************************
**DISTRIBUTED MECHANICAL PROPERTIES ********************************************
********************************************************************************
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
********************************************************************************
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
$MESH
********************************************************************************
**BOUNDARY CONDITIONS **********************************************************
********************************************************************************
$BOUNDARY_CONDITIONS
*End part
********************************************************************************
** ASSEMBLY ********************************************************************
********************************************************************************
*Assembly, name = Assembly
*Instance, name=iSample, part=pSample
*End Instance
*End Assembly
********************************************************************************
** MATERIALS *******************************************************************
********************************************************************************
#MATERIALS
********************************************************************************
** STEPS ***********************************************************************
********************************************************************************
"""


In [297]:
sample = CuboidSample(shape = (2,2,2))
sample.make_mesh()
m = sample.mesh
n = m.nodes
c = n.coords
xm, ym, zm = c.max(axis = 0)
# CORNERS
n[("sets", "pinned")]     = (c.x == 0.) & (c.y == 0.) & (c.z == 0.)
n[("sets", "controlled")] = (c.x == xm) & (c.y == ym) & (c.z == zm)
n[("sets", "zpinned")]    = (c.x == xm) & (c.y == 0.) & (c.z == 0.)

m.nodes.head()
m.elements.materials = ["mat{0}".format(i) for i in m.elements.index]
m.elements.loc[:, ("type", "solver", "") ] = "C3D8"

# BOUNDARY CONDITIONS


equation_pattern = "*EQUATION\n  {0}\n"

bc_type = "periodic" # TEST
if bc_type == "periodic":
    # FACE TO FACE PAIR EQUATIONS
    faces = {"left"   : n.coords[(c.x == 0.)].sort_values(["y", "z"]),
             "right"  : n.coords[(c.x == xm)].sort_values(["y", "z"]),                   
             "bottom" : n.coords[(c.y == 0.)].sort_values(["x", "z"]),
             "top"    : n.coords[(c.y == ym)].sort_values(["x", "z"]),                   
             "back"   : n.coords[(c.z == 0.)].sort_values(["x", "y"]),                   
             "front"  : n.coords[(c.z == zm)].sort_values(["x", "y"]),}

    bc = "\n".join([argiope.utils._unsorted_set(v, k) 
                    for k, v in faces.items()]) + "\n"
    
    bc += argiope.utils._equation(nodes = ["left", "right"], 
                                  dofs = [2,2], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["left", "right"], 
                                  dofs = [3,3], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["bottom", "top"], 
                                  dofs = [1,1], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["bottom", "top"], 
                                  dofs = [3,3], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["back", "front"], 
                                  dofs = [1,1], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["back", "front"], 
                                  dofs = [2,2], 
                                  coefficients = [1., -1.]) + "\n"
    bc += argiope.utils._equation(nodes = ["left0", "pinned"], 
                                  dofs = [1,1], 
                                  coefficients = [1., -1.]) + "\n"
    n[("sets", "left0")]      = (c.x == 0.) & (n.sets.pinned     == False)
    n[("sets", "right0")]     = (c.x == xm) & (n.sets.controlled == False)
    n[("sets", "bottom0")]    = (c.x == 0.) & (n.sets.pinned     == False)
    n[("sets", "top0")]       = (c.x == ym) & (n.sets.controlled == False)
    n[("sets", "front0")]     = (c.x == 0.) & (n.sets.pinned     == False)
    n[("sets", "back0")]      = (c.x == zm) & (n.sets.controlled == False)
out = Template(main_template).substitute(
    MESH = sample.mesh.write_inp(),
    BOUNDARY_CONDITIONS = bc.strip("\n"),
    )

    
print(out)


********************************************************************************
**DISTRIBUTED MECHANICAL PROPERTIES ********************************************
********************************************************************************
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
********************************************************************************
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
********************************************************************************
** NODES ***********************************************************************
********************************************************************************
*NODE
  1, 0.0, 0.0, 0.0
  2, 0.5, 0.0, 0.0
  3, 1.0, 0.0, 0.0
  4, 0.0, 0.5, 0.0
  5, 0.5, 0.5, 0.0
  6, 1.0, 0.5, 0.0
  7, 0.0, 1.0, 0.0
  8, 0.5, 1.0, 0.0
  9, 1.0, 1.0, 0.0
  10, 0.0, 0.0, 0.5
  11, 0.5, 0.0, 0.5
  12, 1.0, 0.0, 0.5
  13, 0.0, 0.5, 0.5
  14, 0.5, 0.5, 0.5
  15, 1.0, 0.5, 0.5
  16, 0.0, 1.0, 0.5
  

In [298]:
faces["top"][n.sets.controlled == False]


/home/lcharleux/Documents/Informatique/anaconda/envs/myconda1/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,x,y,z
node,,,
7,0.0,1.0,0.0
16,0.0,1.0,0.5
25,0.0,1.0,1.0
8,0.5,1.0,0.0
17,0.5,1.0,0.5
26,0.5,1.0,1.0
9,1.0,1.0,0.0
18,1.0,1.0,0.5


In [152]:
m.elements


conn                             materials  sets    type       
          n0  n1  n2  n3  n4  n5  n6  n7             all argiope solver
                                                                       
element                                                                
1          1   2   5   4  10  11  14  13      mat1  True   hexa8       
2          2   3   6   5  11  12  15  14      mat2  True   hexa8       
3          4   5   8   7  13  14  17  16      mat3  True   hexa8       
4          5   6   9   8  14  15  18  17      mat4  True   hexa8       
5         10  11  14  13  19  20  23  22      mat5  True   hexa8       
6         11  12  15  14  20  21  24  23      mat6  True   hexa8       
7         13  14  17  16  22  23  26  25      mat7  True   hexa8       
8         14  15  18  17  23  24  27  26      mat8  True   hexa8

In [177]:
30. * 41

1230.0

In [178]:
20 * 41

820

*EQUATION
  2
  TOP, 1, 1.0
  5, 1, 1.0


In [254]:
def list_to_string(l = range(200), width = 40, indent = "  "):
    """
    Converts a list-like to string with given line width.
    """
    l = [str(v) + "," for v in l]
    counter = 0
    out = "" + indent
    for w in l:
        s = len(w)
        if counter + s > width: 
            out += "\n" + indent
            counter = 0
        out += w
        counter += s
    return out.strip(",")


print(list_to_string())

  0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  16,17,18,19,20,21,22,23,24,25,26,27,28,
  29,30,31,32,33,34,35,36,37,38,39,40,41,
  42,43,44,45,46,47,48,49,50,51,52,53,54,
  55,56,57,58,59,60,61,62,63,64,65,66,67,
  68,69,70,71,72,73,74,75,76,77,78,79,80,
  81,82,83,84,85,86,87,88,89,90,91,92,93,
  94,95,96,97,98,99,100,101,102,103,104,
  105,106,107,108,109,110,111,112,113,114,
  115,116,117,118,119,120,121,122,123,124,
  125,126,127,128,129,130,131,132,133,134,
  135,136,137,138,139,140,141,142,143,144,
  145,146,147,148,149,150,151,152,153,154,
  155,156,157,158,159,160,161,162,163,164,
  165,166,167,168,169,170,171,172,173,174,
  175,176,177,178,179,180,181,182,183,184,
  185,186,187,188,189,190,191,192,193,194,
  195,196,197,198,199


In [262]:
argiope.utils.list_to_string([2])

'  2'